In [1]:
# Pyomo specifics 
from pyomo.environ import *
import pyomo.environ as pe
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
from pyomo.core import Var
from pyomo.core import Constraint
from pyomo.core import Param
import os
import numpy as np
import time
import scipy.io
# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# %matplotlib inline

## Risk-neutral model

In [2]:
# Risk neutral model
def risk_neutral(DataPath=None,
                 mipgap=0.00, 
                 timelimit=3600, 
                 nthreads=4, 
                 solver='cplex', 
                 relaxed=False, 
                 displaySolver=False, 
                 warmstart=False,
                 WSOL={}, 
                 verbose=False,
                 fix=False):

    # Create a solver
    opt = SolverFactory(solver)  # Cplex is called, other options: Gurobi, Knitro, Pico, glpk, etc.

    # Create a model
    model = AbstractModel()      # An abstract model is created, then we need to call the data file

    ### Sets
    model.I = Set()                               # nodes   
    model.J = Set()                               # nodes
    model.S = Set()                               # scenarios
    
    ### Parameters
    model.D = Param(model.I, default=0, mutable=True, within=NonNegativeReals,) # Dem[i] demand(exports)
    model.P = Param(model.J, default=0, mutable=True, within=NonNegativeReals,) # P[j] production
    model.G = Param(model.J, default=0, mutable=True, within=NonNegativeReals,) # P[j] max production (first stage)
    model.d = Param(model.I,model.J,default=0, mutable=True, within=NonNegativeReals,) # dist[i.j] 
    model.pi = Param(default=0, mutable=True, within=NonNegativeReals,) # cost of loss demand > 1 penalise more 
    model.gamma = Param(default=1, mutable=True, within=NonNegativeReals,) # shrinkage factor for inventory
    model.zeta = Param(default=0.1, mutable=True, within=NonNegativeReals,) # shrinkage factor for production ( end point inventory levels multiplier)
    model.N = Param(default=0, mutable=True, within=NonNegativeReals,) # number of nodes
    model.c = Param(model.I,model.J,default=0, mutable=True, within=NonNegativeReals,) # total edge cost(cost of maintaining edge + transport)
    model.prob = Param(model.S, default=0, mutable=True, within=NonNegativeReals,) #beta - capacity multipler
    model.betas = Param(model.J,model.S, default=0, mutable=True, within=NonNegativeReals,)
    model.Ci = Param(model.J, default=1, mutable=True, within=NonNegativeReals,)
    
    ## stage 1 decision variables
    model.W = Var(model.I,model.J, within=NonNegativeReals)     # allocation variable X[i,j,s]
#     model.k = Var(model.I,  within=NonNegativeReals)
    model.In = Var(model.J,  within=NonNegativeReals)
    model.Co = Var(model.I, within=NonNegativeReals)
    
    ### stage 2 Decision Variables
    model.X = Var(model.I,model.J, model.S, within=NonNegativeReals)     # allocation variable X[i,j,s]
    model.C = Var(model.I, model.S, within=NonNegativeReals)
    model.temp = Var(within=NonNegativeReals)
    model.ScenSol = Var(model.S, within=NonNegativeReals)
    model.In2 = Var(model.J, model.S, within=NonNegativeReals)
    
    
    
    ## Quadratic model
    def Mincost(model):
        return model.temp
    model.Mincost = Objective(sense=minimize, rule=Mincost)     
    
    # Objective function definition 
    def AUXM(model):
        return model.temp == sum([model.prob[s] * sum([model.c[i,j] * model.X[i,j,s] for j in model.J for i in model.I]) +\
                                  model.prob[s] * sum([model.Ci[j] * model.In2[j,s] for j in model.J if j > 0]) +\
                                  model.prob[s] * sum([model.X[i,j,s] * model.C[i,s] for i in model.I for j in model.J]) +\
                                  model.prob[s] * sum([model.pi * model.D[i] * model.X[i,0,s] * len(model.J) for i in model.I]) \
                                  for s in model.S]) + sum([model.c[i,j] * model.W[i,j] for j in model.J for i in model.I]) +\
                                  sum([model.W[i,j] * model.Co[i] for i in model.I for j in model.J]) +\
                                  sum([model.Ci[j] * model.In[j] for j in model.J if j > 0]) +\
                                  sum([model.pi * model.D[i] * model.W[i,0] * len(model.J) for i in model.I]) 
    model.AUXM = Constraint(rule=AUXM)
        
    # Constraints
    # define C for stage 2
    def CIS(model, i, s):
        return model.C[i,s] == sum([model.X[i,j,s] for j in model.J if j > 0])
    model.CIS = Constraint(model.I, model.S, rule=CIS) 
    
    # define Co for stage 1
    def CIS2(model, i):
        return model.Co[i] == sum([model.W[i,j] for j in model.J if j > 0])
    model.CIS2 = Constraint(model.I, rule=CIS2) 
        
    # Capacity for stage 1
    def Capacity(model,j):
        return  model.G[j] == sum([ model.W[i,j] * model.D[i] for i in model.I if j>0])+ model.In[j] 
    model.Capacity = Constraint(model.J, rule=Capacity)
    
    # Inventory balance  - stage 2
    def Inventory(model,j,s):
        return  model.betas[j,s]*model.P[j] + model.gamma* model.In[j] == sum([ model.X[i,j,s] * model.D[i] for i in model.I if j>0]) +\
                model.In2[j,s]
    model.Inventory = Constraint(model.J, model.S, rule=Inventory)
    
    # Allocation for stage 2
    def allocation1(model, i):
        return sum([model.W[i,j] for j in model.J]) == 1
    model.allocation1 = Constraint(model.I,  rule= allocation1)
    
    # Allocation for stage 2
    def allocation2(model, i, s):
        return sum([model.X[i,j,s] for j in model.J]) == 1
    model.allocation2 = Constraint(model.I, model.S, rule= allocation2)
    
    # inventory min levels for stage 1
#     def waste1(model,j):
#         return   model.In[j] >= sum([model.D[i] * model.W[i,0]  for i in model.I]
#     model.Capacity = Constraint(model.J, rule=Capacity)
    
    # Inventory floor - stage 2
    def waste2(model,j,s):
        return  model.In2[j,s] >=   model.zeta* model.P[j]
    model.waste2 = Constraint(model.J, model.S, rule=Inventory)
    

     # Scen Sol --> need to update this.
    def ScenSolDef(model, s):
        return model.ScenSol[s] == sum([model.c[i,j] * model.X[i,j,s] for j in model.J for i in model.I])+\
                                   sum([model.Ci[j] * model.In2[j,s] for j in model.J if j > 0]) +\
                                   sum([model.X[i,j,s] * model.C[i,s] for i in model.I for j in model.J]) +\
                                   sum([model.pi * model.D[i] * model.X[i,0,s] * len(model.J) for i in model.I]) +\
                                   sum([model.c[i,j] * model.W[i,j] for j in model.J for i in model.I])  +\
                                   sum([model.W[i,j] * model.Co[i] for i in model.I for j in model.J]) +\
                                   sum([model.Ci[j] * model.In[j] for j in model.J if j > 0]) +\
                                   sum([model.pi * model.D[i] * model.W[i,0] * len(model.J) for i in model.I])                
                              
    model.ScenSolDef = Constraint(model.S, rule=ScenSolDef)
       
    
    # Create a model instance and optimize
    # If none, data should be passed through the python environment (not yet implemented)
    if DataPath is None:
        instance = model.create_instance()
    else:
        instance = model.create_instance(DataPath)
    
     # Fixed variables
    if fix:
        for s in instance.S:
            print('s:',s)
            for k in WSOL['X']:
                instance.X[k[0],k[1],s].fix(WSOL['X'][k])
    
   
    ### Solve the model
    startT = time.time()
    results = opt.solve(instance, tee=displaySolver)
    instance.solutions.load_from(results)
    endT = time.time()    

    # Solution
    SOL = {}
    SOL['X'] = {}
    SOL['W'] = {}
    SOL['In'] = {}
    SOL['In2'] = {}
    SOL['C'] = {}
    SOL['Co'] = {}
    SOL['OBJ'] = []
    SOL['SSOL'] = {}
    SOL['STIME'] = 0
    
    # Saving loop 
    for j in instance.J:
        SOL['In'][j] = instance.In[j].value
        for s in instance.S:
            SOL['In2'][j,s] = instance.In2[j,s].value
            
        
    for s in instance.S:
        SOL['SSOL'][s] = instance.ScenSol[s].value
        for i in instance.I:
            if instance.C[i,s].value > 1e-6:
                SOL['C'][(i,s)] = instance.C[i,s].value
            if instance.Co[i].value > 1e-6:
                SOL['Co'][(i)] = instance.Co[i].value
            for j in instance.J:
                if instance.X[i,j,s].value > 1e-6:
                    SOL['X'][(i,j,s)] = instance.X[i,j,s].value
                if instance.W[i,j].value > 1e-6:
                    SOL['W'][(i,j)] = instance.W[i,j].value
    SOL['OBJ'] = instance.temp.value   
    
    # Print Results
    if verbose:
        print("\n", "-" * 10, "X values", "-" * 10)
    xlist =[]
    for s in instance.S:
        temp_=[]
        for i in instance.I:
            temp = []
            for j in instance.J:
                temp.append(instance.X[i,j,s].value)
                if instance.X[i,j,s].value > 1e-6 and verbose:
                    print("X[", i,j,s, "] =", instance.X[i,j,s].value)
            temp_.append(temp)
        xlist.append(temp_)
        
    if verbose:
        print("\n", "-" * 10, "W values", "-" * 10)
    wlist =[]
    for i in instance.I:
        temp = []
        for j in instance.J:
            temp.append(instance.W[i,j].value)
            if instance.W[i,j].value > 1e-6 and verbose:
                print("W[", i,j, "] =", instance.W[i,j].value)
#         temp_.append(temp)
        wlist.append(temp)
    
    if verbose:
        print("\n", "-" * 10, "In values", "-" * 10)
    In_list =[]
    for j in instance.J:
        In_list.append(instance.In[j].value)
        if instance.In[j].value > 1e-6 and verbose:
            print("In[", j, "] =", instance.In[j].value)

    
    if verbose:
        print("\n", "-" * 10, "In2 values", "-" * 10)
    In2_list =[]
    for s in instance.S:
        temp = []
        for j in instance.J:
            temp.append(instance.In2[j,s].value)
            if instance.In2[j,s].value > 1e-6 and verbose:
                print("In2[", j,s, "] =", instance.In2[j,s].value)
        In2_list.append(temp)

    # Total objective function
    print("\n", "-" * 10, "Objective function", "-" * 10)
    print("Min Cost:", value(instance.Mincost))
    print("Total Solving time:", np.round(endT - startT, 3), "[s]")
    SOL['STIME'] = np.round(endT - startT, 3)
    
    return SOL

In [3]:
SOL_ = risk_neutral(DataPath= '../stochastic/scens_two_stage/deterministic/deterministic.dat',
                 mipgap=0.00, 
                 timelimit=3600, 
                 nthreads=4, 
                 solver='ipopt', 
                 relaxed=False, 
                 displaySolver=False, 
                 warmstart=False,
                 WSOL={}, 
                 verbose=True,
                 fix=False)
SOL_
name = 'Results_RN_twostage/RN_SOL_deterministic.pickle'
with open(name, 'wb') as handle:
    pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)


 ---------- X values ----------
X[ 1 1 0 ] = 0.015108458264638861
X[ 1 2 0 ] = 0.015107060168786612
X[ 1 3 0 ] = 0.01510413121159347
X[ 1 4 0 ] = 0.01510073289558635
X[ 1 5 0 ] = 0.015097574306245545
X[ 1 6 0 ] = 0.015095432754435466
X[ 1 7 0 ] = 0.015092390144720944
X[ 1 8 0 ] = 0.015090330205358469
X[ 1 9 0 ] = 0.015089455708009139
X[ 1 10 0 ] = 0.015086062157282072
X[ 1 11 0 ] = 0.015084870982819838
X[ 1 12 0 ] = 0.0150831316642067
X[ 1 13 0 ] = 0.015061066588149851
X[ 1 14 0 ] = 0.015061066603230288
X[ 1 15 0 ] = 0.014976953677614452
X[ 1 16 0 ] = 0.015045274956793434
X[ 1 17 0 ] = 0.015030689607837946
X[ 1 18 0 ] = 0.015002880057013045
X[ 1 19 0 ] = 0.014980579221845697
X[ 1 20 0 ] = 0.014980579222680854
X[ 1 21 0 ] = 0.01496949161440211
X[ 1 22 0 ] = 0.014949567684122193
X[ 1 23 0 ] = 0.014934252471115607
X[ 1 24 0 ] = 0.014883599699624446
X[ 1 25 0 ] = 0.014803183994617594
X[ 1 26 0 ] = 0.014759230002021034
X[ 1 27 0 ] = 0.014758162218822547
X[ 1 28 0 ] = 0.014731569134317462
X

X[ 11 28 0 ] = 0.0175618650791498
X[ 11 29 0 ] = 0.017561865074386923
X[ 11 30 0 ] = 0.01766708166059632
X[ 11 31 0 ] = 0.01652382922513846
X[ 11 32 0 ] = 0.016507010898283114
X[ 11 33 0 ] = 0.016233238755743746
X[ 11 34 0 ] = 0.01621519160973266
X[ 11 35 0 ] = 0.016215191614671803
X[ 11 36 0 ] = 0.015914428163620497
X[ 11 37 0 ] = 0.015892455061590816
X[ 11 38 0 ] = 0.015539777061302315
X[ 11 39 0 ] = 0.013981939506405796
X[ 11 40 0 ] = 0.01320388953897583
X[ 11 41 0 ] = 0.013108248707825308
X[ 11 42 0 ] = 0.013108248705473686
X[ 11 43 0 ] = 0.010419738425638592
X[ 11 44 0 ] = 0.012020956588878218
X[ 11 45 0 ] = 0.010810798260373547
X[ 11 46 0 ] = 0.010810798262809562
X[ 11 47 0 ] = 0.010810798259716469
X[ 11 48 0 ] = 0.009416862280386925
X[ 11 49 0 ] = 0.009127175981179568
X[ 11 50 0 ] = 0.009127175980672605
X[ 11 51 0 ] = 0.00912717598396918
X[ 11 52 0 ] = 0.009127175986865797
X[ 11 53 0 ] = 0.00912717598219415
X[ 11 54 0 ] = 0.009127175981491781
X[ 11 55 0 ] = 0.009127175987403967


X[ 27 22 0 ] = 0.009956668411299122
X[ 27 23 0 ] = 0.009150442011879433
X[ 27 24 0 ] = 0.007210779293403736
X[ 27 25 0 ] = 0.005383153530835445
X[ 27 26 0 ] = 0.00472329327846388
X[ 27 27 0 ] = 0.004709227345120471
X[ 27 28 0 ] = 0.004383503431898902
X[ 27 29 0 ] = 0.004383503432825094
X[ 27 30 0 ] = 0.004682832216115266
X[ 27 31 0 ] = 0.002618184414957241
X[ 27 32 0 ] = 0.0026001346770285
X[ 27 33 0 ] = 0.0023335693425582996
X[ 27 34 0 ] = 0.002317624629026539
X[ 27 35 0 ] = 0.002317624628610394
X[ 27 36 0 ] = 0.002076441652640645
X[ 27 37 0 ] = 0.002060458411555587
X[ 27 38 0 ] = 0.0018290380479726069
X[ 27 39 0 ] = 0.0011695320110949475
X[ 27 40 0 ] = 0.0009650029737807129
X[ 27 41 0 ] = 0.0009433715485010198
X[ 27 42 0 ] = 0.0009433715485282076
X[ 27 43 0 ] = 0.0005245331352344503
X[ 27 44 0 ] = 0.0007370589319660409
X[ 27 45 0 ] = 0.0005690957263861803
X[ 27 46 0 ] = 0.0005690957263600749
X[ 27 47 0 ] = 0.0005690957263657832
X[ 27 48 0 ] = 0.00042626312483323687
X[ 27 49 0 ] = 0.0

X[ 41 82 0 ] = 0.0006230297610134588
X[ 41 83 0 ] = 0.0006230297610197924
X[ 42 1 0 ] = 0.04261344208282371
X[ 42 2 0 ] = 0.04228233843550854
X[ 42 3 0 ] = 0.04160491603841014
X[ 42 4 0 ] = 0.04084532760904293
X[ 42 5 0 ] = 0.040163485609426175
X[ 42 6 0 ] = 0.03971384286476659
X[ 42 7 0 ] = 0.039091839507165264
X[ 42 8 0 ] = 0.03868153291166021
X[ 42 9 0 ] = 0.03850990842926024
X[ 42 10 0 ] = 0.03785789690854959
X[ 42 11 0 ] = 0.03763417005667009
X[ 42 12 0 ] = 0.03731213777710812
X[ 42 13 0 ] = 0.03365319682139732
X[ 42 14 0 ] = 0.03365319687311608
X[ 42 15 0 ] = 0.024448907258716564
X[ 42 16 0 ] = 0.0314410426708363
X[ 42 17 0 ] = 0.02963800345859414
X[ 42 18 0 ] = 0.02670931723083485
X[ 42 19 0 ] = 0.0247421580374092
X[ 42 20 0 ] = 0.02474215805151019
X[ 42 21 0 ] = 0.023866276326736307
X[ 42 22 0 ] = 0.02243609081035401
X[ 42 23 0 ] = 0.02144589021972866
X[ 42 24 0 ] = 0.018703662120261793
X[ 42 25 0 ] = 0.015524314418248074
X[ 42 26 0 ] = 0.014194381885099956
X[ 42 27 0 ] = 0.014

X[ 55 67 0 ] = 0.0022010485643096096
X[ 55 68 0 ] = 0.002201048564368262
X[ 55 69 0 ] = 0.0022010485643015995
X[ 55 70 0 ] = 0.002201048564244112
X[ 55 71 0 ] = 0.0022010485642786435
X[ 55 72 0 ] = 0.0022010485642820887
X[ 55 73 0 ] = 0.002201048564246519
X[ 55 74 0 ] = 0.0007983731217505674
X[ 55 75 0 ] = 0.002201048564425762
X[ 55 76 0 ] = 0.002201048564420307
X[ 55 77 0 ] = 0.0022010485641406627
X[ 55 78 0 ] = 0.0022010485643603104
X[ 55 79 0 ] = 0.00220104856423406
X[ 55 80 0 ] = 0.0022010485641135212
X[ 55 81 0 ] = 0.002201048564224177
X[ 55 82 0 ] = 0.0022010485640504727
X[ 55 83 0 ] = 0.0022010485644632983
X[ 56 1 0 ] = 0.03126420618744928
X[ 56 2 0 ] = 0.0311806079709071
X[ 56 3 0 ] = 0.0310068658378158
X[ 56 4 0 ] = 0.0308076085850002
X[ 56 5 0 ] = 0.030624611037752027
X[ 56 6 0 ] = 0.030501726778853187
X[ 56 7 0 ] = 0.030328764744177845
X[ 56 8 0 ] = 0.030212734774647407
X[ 56 9 0 ] = 0.03016373604672906
X[ 56 10 0 ] = 0.029975033402529317
X[ 56 11 0 ] = 0.029909335334505825


...

W[ 58 7 ] = 0.025881506960888923
W[ 58 8 ] = 0.025844355931414136
W[ 58 9 ] = 0.025778521795263434
W[ 58 10 ] = 0.025735427191307808
W[ 58 11 ] = 0.025716674591947354
W[ 58 12 ] = 0.025665512738894124
W[ 58 13 ] = 0.02514711750182015
W[ 58 14 ] = 0.025147117501821525
W[ 58 15 ] = 0.024956882881744035
W[ 58 16 ] = 0.02482403438669354
W[ 58 17 ] = 0.024456963677011907
W[ 58 18 ] = 0.023586467276556472
W[ 58 19 ] = 0.02337856719574807
W[ 58 20 ] = 0.02337856719574246
W[ 58 21 ] = 0.023149988110596317
W[ 58 22 ] = 0.022452805051732652
W[ 58 23 ] = 0.022447069841289483
W[ 58 24 ] = 0.021897441345805093
W[ 58 25 ] = 0.020124464814888166
W[ 58 26 ] = 0.01977253984286779
W[ 58 27 ] = 0.01942159475417394
W[ 58 28 ] = 0.019026634444059824
W[ 58 29 ] = 0.019026634444085505
W[ 58 30 ] = 0.01862948526594705
W[ 58 31 ] = 0.016441637315618367
W[ 58 32 ] = 0.015828322375841388
W[ 58 33 ] = 0.015114836181843675
W[ 58 34 ] = 0.014377426219058569
W[ 58 35 ] = 0.014377426219041945
W[ 58 36 ] = 0.014334741

W[ 76 63 ] = 0.0003528210171860028
W[ 76 64 ] = 0.00016126396687884658
W[ 76 65 ] = 0.00016126396687884072
W[ 76 66 ] = 0.0001612639668781163
W[ 76 67 ] = 0.00016126396687871382
W[ 76 68 ] = 0.00016126396687911733
W[ 76 69 ] = 0.00016126396687853723
W[ 76 70 ] = 0.00016126396687861613
W[ 76 71 ] = 0.0001612639668784425
W[ 76 72 ] = 0.00016126396687832695
W[ 76 73 ] = 0.00016126396687686623
W[ 76 74 ] = 7.808562140039991e-05
W[ 76 75 ] = 0.00016126396687918168
W[ 76 76 ] = 0.00016126396687904984
W[ 76 77 ] = 0.00016126396687807086
W[ 76 78 ] = 0.00016126396687627902
W[ 76 79 ] = 0.00016126396687606657
W[ 76 80 ] = 0.00016126396687889463
W[ 76 81 ] = 0.00016126396687904116
W[ 76 82 ] = 0.00016126396687810818
W[ 76 83 ] = 0.0001612639668762373
W[ 77 0 ] = 1.0000000066465236
W[ 78 0 ] = 1.0000000066465236
W[ 79 0 ] = 1.0000000066465236
W[ 80 1 ] = 0.1863725224967256
W[ 80 2 ] = 0.14766163060926715
W[ 80 3 ] = 0.10410785949309172
W[ 80 4 ] = 0.0755705856547214
W[ 80 5 ] = 0.0593600384977657

W[ 92 31 ] = 0.009897234739029617
W[ 92 32 ] = 0.009174970205008653
W[ 92 33 ] = 0.008399312922654147
W[ 92 34 ] = 0.007662258994264131
W[ 92 35 ] = 0.007662258994248194
W[ 92 36 ] = 0.007621439044330898
W[ 92 37 ] = 0.006886014574880645
W[ 92 38 ] = 0.006843937231069554
W[ 92 39 ] = 0.005224873564969775
W[ 92 40 ] = 0.004460644219374932
W[ 92 41 ] = 0.0038006731522572763
W[ 92 42 ] = 0.003800673152243061
W[ 92 43 ] = 0.0031264750103588545
W[ 92 44 ] = 0.0024113291214888375
W[ 92 45 ] = 0.0023749712690560847
W[ 92 46 ] = 0.0023749712690443766
W[ 92 47 ] = 0.002374971269044284
W[ 92 48 ] = 0.0022564224810021865
W[ 92 49 ] = 0.0017048268127219071
W[ 92 50 ] = 0.0017048268127149541
W[ 92 51 ] = 0.001704826812717068
W[ 92 52 ] = 0.0017048268127072635
W[ 92 53 ] = 0.001704826812725369
W[ 92 54 ] = 0.0017048268127247987
W[ 92 55 ] = 0.0017048268127154477
W[ 92 56 ] = 0.0008784632394178795
W[ 92 57 ] = 0.0008784632394633206
W[ 92 58 ] = 0.0010741890562937708
W[ 92 59 ] = 0.001074189056296476


W[ 108 35 ] = 0.013387814522511577
W[ 108 36 ] = 0.013341573108240926
W[ 108 37 ] = 0.012477587456632536
W[ 108 38 ] = 0.012426315139822968
W[ 108 39 ] = 0.010283433545830455
W[ 108 40 ] = 0.009141732677983454
W[ 108 41 ] = 0.008077130090048696
W[ 108 42 ] = 0.008077130090024918
W[ 108 43 ] = 0.006905111017287574
W[ 108 44 ] = 0.0055569923702523585
W[ 108 45 ] = 0.005485318768880106
W[ 108 46 ] = 0.005485318768856974
W[ 108 47 ] = 0.00548531876885679
W[ 108 48 ] = 0.005249399228090529
W[ 108 49 ] = 0.0041050020209667795
W[ 108 50 ] = 0.004105002020951849
W[ 108 51 ] = 0.004105002020956388
W[ 108 52 ] = 0.004105002020935334
W[ 108 53 ] = 0.004105002020974213
W[ 108 54 ] = 0.004105002020972988
W[ 108 55 ] = 0.004105002020952908
W[ 108 56 ] = 0.002232308655953401
W[ 108 57 ] = 0.0022323086556205202
W[ 108 58 ] = 0.002694352593561215
W[ 108 59 ] = 0.0026943525935675186
W[ 108 60 ] = 0.0027489108341657836
W[ 108 61 ] = 0.0026943525935515518
W[ 108 62 ] = 0.0026943525935659443
W[ 108 63 ] = 

W[ 143 42 ] = 0.01343612362343939
W[ 143 43 ] = 0.01306717812301298
W[ 143 44 ] = 0.012493607884425498
W[ 143 45 ] = 0.012457013166161894
W[ 143 46 ] = 0.012457013166149963
W[ 143 47 ] = 0.01245701316614987
W[ 143 48 ] = 0.012331158503012621
W[ 143 49 ] = 0.011573258046924256
W[ 143 50 ] = 0.011573258046912388
W[ 143 51 ] = 0.011573258046915997
W[ 143 52 ] = 0.011573258046899263
W[ 143 53 ] = 0.011573258046930164
W[ 143 54 ] = 0.011573258046929191
W[ 143 55 ] = 0.01157325804691323
W[ 143 56 ] = 0.009359602961795027
W[ 143 57 ] = 0.009359602961986074
W[ 143 58 ] = 0.010084693199308364
W[ 143 59 ] = 0.010084693199317193
W[ 143 60 ] = 0.010160168802409282
W[ 143 61 ] = 0.010084693199294826
W[ 143 62 ] = 0.010084693199314989
W[ 143 63 ] = 0.010084693199343086
W[ 143 64 ] = 0.007182019895531261
W[ 143 65 ] = 0.007182019895531121
W[ 143 66 ] = 0.007182019895514019
W[ 143 67 ] = 0.007182019895528126
W[ 143 68 ] = 0.007182019895537652
W[ 143 69 ] = 0.007182019895523956
W[ 143 70 ] = 0.00718201

In [4]:
## Running Experiments for individual scenarios
nscen = [i for i in range(1,183)]
for ii in nscen:
    SOL_ = risk_neutral(DataPath='scens_two_stage/deterministic/scen{}.dat'.format(ii),
                        mipgap=0.00, 
                        timelimit=3600, 
                        nthreads=4, 
                        solver="ipopt", 
                        relaxed=False, 
                        displaySolver=True, 
                        warmstart=False,
                        WSOL={},  
                        verbose=False,)
    ## save solutions in pickle file 
    name = 'Results_RN_twostage/individual_scens/RN_det_scen{}'.format(ii) +'.pickle'
    with open(name, 'wb') as handle:
        pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   313047
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    66612

Total number of variables............................:    67344
                     variables with only lower bounds:    67344
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

   1  1.1319302e-02 1.43e+02 2.27e+00  -1.0 1.74e+00    -  6.00e-03 1.38e-02h  1
   2  1.2529030e-02 1.42e+02 2.26e+00  -1.0 1.72e+00    -  1.99e-02 1.27e-02h  1
   3  1.9976918e-02 1.29e+02 2.08e+00  -1.0 1.69e+00    -  4.15e-02 7.80e-02h  1
   4  2.3366424e-02 1.24e+02 6.27e+01  -1.0 1.56e+00    -  2.63e-01 3.56e-02h  1
   5  2.7122454e-02 1.20e+02 6.60e+02  -1.0 1.51e+00    -  2.00e-01 2.66e-02h  1
   6  4.4537775e-02 1.18e+02 1.38e+03  -1.0 1.47e+00    -  3.87e-02 2.03e-02h  1
   7  7.8260580e+00 1.14e+02 1.22e+03  -1.0 2.77e+02    -  1.71e-04 2.81e-02h  1
   8  1.7595666e+02 6.45e+01 1.01e+03  -1.0 5.84e+02    -  4.59e-01 2.88e-01h  1
   9  2.9575114e+02 2.12e+01 5.17e+02  -1.0 2.88e+02    -  2.31e-01 4.15e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.2781288e+02 1.20e+01 3.20e+02  -1.0 1.08e+02    -  1.74e-01 2.98e-01h  1
  11  3.3174004e+02 1.11e+01 4.07e+02  -1.0 5.94e+01    -  3.95e-01 6.61e-02h  1
  12  3.3684408e+02 9.94e+00

   1  1.1319156e-02 1.43e+02 2.27e+00  -1.0 1.74e+00    -  6.00e-03 1.38e-02h  1
   2  1.2528810e-02 1.42e+02 2.26e+00  -1.0 1.72e+00    -  1.99e-02 1.27e-02h  1
   3  1.9974880e-02 1.29e+02 2.08e+00  -1.0 1.69e+00    -  4.15e-02 7.80e-02h  1
   4  2.3362649e-02 1.24e+02 4.79e+01  -1.0 1.56e+00    -  2.10e-01 3.55e-02h  1
   5  2.6838138e-02 1.20e+02 6.42e+02  -1.0 1.51e+00    -  2.32e-01 2.65e-02h  1
   6  4.2717730e-02 1.18e+02 1.47e+03  -1.0 1.47e+00    -  4.18e-02 2.00e-02h  1
   7  7.2610478e+00 1.15e+02 1.14e+03  -1.0 2.59e+02    -  1.76e-04 2.78e-02h  1
   8  1.6695142e+02 6.82e+01 9.82e+02  -1.0 5.88e+02    -  4.46e-01 2.72e-01h  1
   9  2.9564171e+02 2.09e+01 4.84e+02  -1.0 3.02e+02    -  2.41e-01 4.26e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.2653350e+02 1.23e+01 3.05e+02  -1.0 1.08e+02    -  1.68e-01 2.85e-01h  1
  11  3.4379286e+02 7.47e+00 2.52e+02  -1.0 6.15e+01    -  3.90e-01 2.81e-01h  1
  12  3.5000736e+02 5.95e+00

   1  1.1319102e-02 1.43e+02 2.27e+00  -1.0 1.74e+00    -  6.00e-03 1.38e-02h  1
   2  1.2528729e-02 1.42e+02 2.26e+00  -1.0 1.72e+00    -  1.99e-02 1.27e-02h  1
   3  1.9973999e-02 1.29e+02 2.08e+00  -1.0 1.69e+00    -  4.15e-02 7.80e-02h  1
   4  2.3361273e-02 1.24e+02 4.82e+01  -1.0 1.56e+00    -  2.11e-01 3.55e-02h  1
   5  2.6841849e-02 1.20e+02 6.42e+02  -1.0 1.51e+00    -  2.31e-01 2.65e-02h  1
   6  4.2731905e-02 1.18e+02 1.47e+03  -1.0 1.47e+00    -  4.18e-02 2.00e-02h  1
   7  7.2648451e+00 1.15e+02 1.14e+03  -1.0 2.59e+02    -  1.76e-04 2.78e-02h  1
   8  1.6697548e+02 6.82e+01 9.80e+02  -1.0 5.88e+02    -  4.43e-01 2.72e-01h  1
   9  2.9566040e+02 2.10e+01 4.86e+02  -1.0 3.02e+02    -  2.48e-01 4.26e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.2677113e+02 1.22e+01 3.04e+02  -1.0 1.08e+02    -  1.67e-01 2.87e-01h  1
  11  3.4390139e+02 7.45e+00 2.51e+02  -1.0 6.12e+01    -  3.86e-01 2.80e-01h  1
  12  3.5446432e+02 4.55e+00

   3  1.9973534e-02 1.29e+02 2.08e+00  -1.0 1.69e+00    -  4.15e-02 7.80e-02h  1
   4  2.3360623e-02 1.24e+02 4.83e+01  -1.0 1.56e+00    -  2.12e-01 3.55e-02h  1
   5  2.6843423e-02 1.20e+02 6.42e+02  -1.0 1.51e+00    -  2.30e-01 2.65e-02h  1
   6  4.2738181e-02 1.18e+02 1.47e+03  -1.0 1.47e+00    -  4.17e-02 2.00e-02h  1
   7  7.2665438e+00 1.15e+02 1.14e+03  -1.0 2.59e+02    -  1.76e-04 2.78e-02h  1
   8  1.6698872e+02 6.82e+01 9.79e+02  -1.0 5.88e+02    -  4.43e-01 2.72e-01h  1
   9  2.9564614e+02 2.10e+01 4.87e+02  -1.0 3.02e+02    -  2.51e-01 4.26e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.2684868e+02 1.22e+01 3.05e+02  -1.0 1.08e+02    -  1.67e-01 2.88e-01h  1
  11  3.4396313e+02 7.43e+00 2.50e+02  -1.0 6.11e+01    -  3.84e-01 2.80e-01h  1
  12  3.5627806e+02 3.92e+00 2.08e+02  -1.0 3.64e+01    -  5.53e-01 3.38e-01h  1
  13  3.6113185e+02 2.75e+00 8.21e+02  -1.0 2.01e+01    -  7.72e-01 2.41e-01h  1
  14  3.6299958e+02 2.40e+00

...


 ---------- Objective function ----------
Min Cost: 162.07010342405204
Total Solving time: 21.225 [s]
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   313047
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    66612

Total number of variables............................:    67344
                     variables with only lower bounds:    67344
                variables with

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  3.6601316e+02 7.77e-05 3.87e+03  -3.8 7.32e+00    -  6.25e-01 4.91e-01f  2
  21  1.7086987e+02 4.50e-06 1.42e+03  -3.8 1.96e+02    -  2.26e-02 9.95e-01f  1
  22  1.7066887e+02 4.22e-06 1.77e+02  -3.8 3.22e+00    -  1.00e+00 6.25e-02h  5
  23  1.6765384e+02 1.73e-07 2.53e-07  -3.8 3.02e+00    -  1.00e+00 1.00e+00h  1
  24  1.6219613e+02 4.06e-08 5.24e+00  -8.6 5.58e+00    -  9.72e-01 9.78e-01f  1
  25  1.6207010e+02 4.68e-08 2.40e+00  -8.6 1.26e-01    -  1.00e+00 9.98e-01f  1
  26  1.6207011e+02 4.68e-08 1.08e+03  -8.6 9.09e-05    -  1.00e+00 3.12e-02f  6
  27  1.6207010e+02 4.69e-08 1.46e-11  -8.6 6.78e-05    -  1.00e+00 1.00e+00s 22
  28  1.6207010e+02 4.69e-08 1.46e-11  -8.6 5.48e-07    -  1.00e+00 1.00e+00s 22

Number of Iterations....: 28

                                   (scaled)                 (unscaled)
Objective...............:   1.6207010342405204e+02    1.6207010342405204e+02
Dual infeas

   1  1.1318940e-02 1.43e+02 2.27e+00  -1.0 1.74e+00    -  6.00e-03 1.38e-02h  1
   2  1.2528481e-02 1.42e+02 2.26e+00  -1.0 1.72e+00    -  1.99e-02 1.27e-02h  1
   3  1.9970541e-02 1.29e+02 2.08e+00  -1.0 1.69e+00    -  4.15e-02 7.80e-02h  1
   4  2.3357635e-02 1.24e+02 4.88e+01  -1.0 1.56e+00    -  2.13e-01 3.55e-02h  1
   5  2.6848296e-02 1.20e+02 6.42e+02  -1.0 1.51e+00    -  2.29e-01 2.65e-02h  1
   6  4.2759976e-02 1.18e+02 1.46e+03  -1.0 1.47e+00    -  4.17e-02 2.00e-02h  1
   7  7.2725942e+00 1.15e+02 1.14e+03  -1.0 2.60e+02    -  1.75e-04 2.78e-02h  1
   8  1.6705602e+02 6.82e+01 9.83e+02  -1.0 5.88e+02    -  4.45e-01 2.72e-01h  1
   9  2.9547088e+02 2.11e+01 4.97e+02  -1.0 3.02e+02    -  2.65e-01 4.25e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.2715849e+02 1.21e+01 3.13e+02  -1.0 1.09e+02    -  1.76e-01 2.91e-01h  1
  11  3.4454347e+02 7.29e+00 2.52e+02  -1.0 6.09e+01    -  3.87e-01 2.86e-01h  1
  12  3.6021784e+02 2.45e+00


 ---------- Objective function ----------
Min Cost: 162.07010342405204
Total Solving time: 21.823 [s]
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   313047
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    66612

Total number of variables............................:    67344
                     variables with only lower bounds:    67344
                variables with

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  3.6601316e+02 7.77e-05 3.87e+03  -3.8 7.32e+00    -  6.25e-01 4.91e-01f  2
  21  1.7086987e+02 4.50e-06 1.42e+03  -3.8 1.96e+02    -  2.26e-02 9.95e-01f  1
  22  1.7066887e+02 4.22e-06 1.77e+02  -3.8 3.22e+00    -  1.00e+00 6.25e-02h  5
  23  1.6765384e+02 1.73e-07 2.53e-07  -3.8 3.02e+00    -  1.00e+00 1.00e+00h  1
  24  1.6219613e+02 4.06e-08 5.24e+00  -8.6 5.58e+00    -  9.72e-01 9.78e-01f  1
  25  1.6207010e+02 4.68e-08 2.40e+00  -8.6 1.26e-01    -  1.00e+00 9.98e-01f  1
  26  1.6207011e+02 4.68e-08 1.08e+03  -8.6 9.09e-05    -  1.00e+00 3.12e-02f  6
  27  1.6207010e+02 4.69e-08 1.46e-11  -8.6 6.78e-05    -  1.00e+00 1.00e+00s 22
  28  1.6207010e+02 4.69e-08 1.46e-11  -8.6 5.48e-07    -  1.00e+00 1.00e+00s 22

Number of Iterations....: 28

                                   (scaled)                 (unscaled)
Objective...............:   1.6207010342405204e+02    1.6207010342405204e+02
Dual infeas

In [6]:
## Running Experiments for changing gamma ( shrinkage factor)
for ii in [0.75,0.8,0.9,1]:
    SOL_ = risk_neutral(DataPath='scens_two_stage/top_suppliers_scens/test_gamma{}scen.dat'.format(ii),
                        mipgap=0.00, 
                        timelimit=3600, 
                        nthreads=4, 
                        solver="ipopt", 
                        relaxed=False, 
                        displaySolver=True, 
                        warmstart=False,
                        WSOL={},  
                        verbose=False,)
    ## save solutions in pickle file 
    name = 'Results_RN_twostage/individual_scens/RN_gamma_scen{}'.format(ii) +'.pickle'
    with open(name, 'wb') as handle:
        pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1104403
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   199836

Total number of variables............................:   202032
                     variables with only lower bounds:   202032
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

  77  7.8274170e+02 4.69e-08 2.69e+02  -8.6 1.43e-02  -4.5 1.00e+00 9.23e-01f  1
  78  7.8274163e+02 4.69e-08 3.26e+03  -8.6 1.06e-03  -5.0 1.00e+00 7.31e-02f  1
  79  7.8274099e+02 4.69e-08 7.73e+02  -8.6 8.70e-04  -5.4 1.00e+00 7.80e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  7.8274094e+02 4.69e-08 2.49e+03  -8.6 1.75e-04  -5.0 1.00e+00 2.92e-01f  1
  81  7.8274094e+02 4.69e-08 3.44e+03  -8.6 1.16e-04  -4.6 1.00e+00 2.06e-02f  6
  82  7.8274083e+02 4.69e-08 2.94e+02  -8.6 1.12e-04  -5.1 1.00e+00 9.57e-01H  1
  83  7.8274083e+02 4.69e-08 5.82e-11  -8.6 1.10e-05  -5.5 1.00e+00 1.00e+00f  1
  84  7.8274083e+02 4.69e-08 5.82e-11  -8.6 2.40e-06  -5.1 1.00e+00 1.00e+00h  1
  85  7.8274083e+02 4.69e-08 4.37e-11  -8.6 2.53e-06  -5.6 1.00e+00 1.00e+00h  1
  86  7.8274083e+02 4.69e-08 4.37e-11  -8.6 7.65e-07  -5.2 1.00e+00 1.00e+00h  1
  87  7.8274083e+02 4.69e-08 3.64e-11  -8.6 2.83e-07  -4.7 1.00e+00 1.00e+00h  1
  88  7.8274083e+02 4.69e-08

  43  9.9716594e+02 1.73e-04 9.22e+02  -3.8 1.19e+02    -  8.15e-02 4.22e-02f  1
  44  9.9086996e+02 1.64e-04 9.00e+02  -3.8 1.21e+02    -  9.06e-02 5.64e-02f  1
  45  9.8522092e+02 1.55e-04 1.10e+03  -3.8 1.14e+02    -  3.77e-01 5.35e-02f  1
  46  9.7968442e+02 1.47e-04 1.12e+03  -3.8 1.12e+02    -  2.20e-01 5.38e-02f  1
  47  9.7372406e+02 1.38e-04 1.09e+03  -3.8 1.11e+02    -  1.53e-01 5.86e-02f  1
  48  9.6362552e+02 1.30e-04 1.12e+03  -3.8 1.94e+02    -  3.57e-01 5.49e-02f  1
  49  9.5415978e+02 1.22e-04 1.06e+03  -3.8 1.58e+02    -  9.92e-02 6.42e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  9.4499997e+02 1.15e-04 1.04e+03  -3.8 1.70e+02    -  3.14e-01 5.74e-02f  1
  51  9.3396938e+02 1.06e-04 9.84e+02  -3.8 1.60e+02    -  2.67e-01 7.38e-02f  1
  52  9.2563276e+02 1.00e-04 9.39e+02  -3.8 1.49e+02    -  2.44e-01 5.99e-02f  1
  53  9.1981254e+02 9.57e-05 9.07e+02  -3.8 1.40e+02    -  2.29e-01 4.47e-02f  1
  54  9.1072851e+02 8.86e-05

   1  1.0671433e-02 1.44e+02 1.60e+00  -1.0 2.44e+00    -  4.22e-03 6.80e-03h  1
   2  1.1260216e-02 1.44e+02 3.17e+00  -1.0 1.88e+00    -  1.24e-02 5.94e-03h  1
   3  1.4723021e-02 1.38e+02 1.54e+00  -1.0 1.87e+00    -  2.65e-02 3.47e-02h  1
   4  2.3041379e-02 1.25e+02 1.62e+00  -1.0 1.80e+00    -  8.74e-02 8.19e-02h  1
   5  2.5703809e-02 1.22e+02 1.77e+02  -1.0 1.65e+00    -  2.20e-01 2.41e-02h  1
   6  2.9467266e-02 1.19e+02 4.31e+03  -1.0 1.61e+00    -  3.87e-01 2.08e-02h  1
   7  1.4854301e+00 1.17e+02 3.41e+02  -1.0 1.50e+02    -  2.15e-04 1.18e-02h  1
   8  1.9230254e+02 5.47e+01 2.81e+03  -1.0 5.63e+02    -  1.63e-03 3.40e-01h  1
   9  2.4118383e+02 4.26e+01 2.43e+03  -1.0 2.89e+02    -  2.57e-01 1.70e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  2.6336851e+02 3.69e+01 2.68e+03  -1.0 1.92e+02    -  3.18e-01 1.16e-01h  1
  11  2.7566610e+02 3.38e+01 3.03e+03  -1.0 1.61e+02    -  3.13e-01 7.73e-02h  1
  12  2.8198596e+02 3.23e+01

  94  7.6470144e+02 4.69e-08 5.82e-11  -8.6 1.39e-06  -5.2 1.00e+00 1.00e+00h  1
  95  7.6470144e+02 4.69e-08 5.09e-11  -8.6 3.90e-07  -4.8 1.00e+00 1.00e+00h  1
  96  7.6470144e+02 4.69e-08 3.64e-11  -8.6 1.38e-07  -4.4 1.00e+00 1.00e+00h  1
  97  7.6470144e+02 4.69e-08 4.37e-11  -8.6 3.81e-07  -4.8 1.00e+00 1.00e+00h  1
  98  7.6470144e+02 4.69e-08 5.09e-11  -8.6 1.78e-07  -4.4 1.00e+00 1.00e+00h  1
  99  7.6470144e+02 4.69e-08 5.82e-11  -8.6 3.82e-07  -4.9 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  7.6470143e+02 4.69e-08 5.82e-11  -8.6 9.31e-07  -5.4 1.00e+00 1.00e+00h  1
 101  7.6470143e+02 4.69e-08 4.37e-11  -8.6 3.22e-07  -4.9 1.00e+00 1.00e+00h  1
 102  7.6470143e+02 4.69e-08 4.37e-11  -8.6 8.00e-07  -5.4 1.00e+00 1.00e+00h  1
 103  7.6470143e+02 4.69e-08 5.82e-11  -8.6 4.30e-07  -5.0 1.00e+00 1.00e+00h  1

Number of Iterations....: 103

                                   (scaled)                 (unscaled)
Object

  54  8.5618652e+02 2.43e-05 1.16e+03  -3.8 1.08e+02    -  3.21e-01 9.01e-02f  1
  55  8.4982018e+02 2.26e-05 1.08e+03  -3.8 9.77e+01    -  2.48e-01 7.02e-02f  1
  56  8.4205293e+02 2.06e-05 9.86e+02  -3.8 9.03e+01    -  2.56e-01 9.21e-02f  1
  57  8.3576969e+02 1.89e-05 9.07e+02  -3.8 8.16e+01    -  3.32e-01 8.24e-02f  1
  58  8.2953275e+02 1.72e-05 8.26e+02  -3.8 7.70e+01    -  2.27e-01 9.01e-02f  1
  59  8.2520501e+02 1.60e-05 7.70e+02  -3.8 6.95e+01    -  4.62e-01 6.89e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60  8.1804766e+02 1.40e-05 6.77e+02  -3.8 6.39e+01    -  3.77e-01 1.21e-01f  1
  61  8.0656845e+02 1.09e-05 5.27e+02  -3.8 5.62e+01    -  4.23e-01 2.22e-01f  1
  62  8.0225759e+02 9.76e-06 4.71e+02  -3.8 4.34e+01    -  4.13e-01 1.07e-01f  1
  63  7.8985373e+02 6.40e-06 3.09e+02  -3.8 3.90e+01    -  5.70e-01 3.45e-01f  1
  64  7.8741324e+02 5.74e-06 2.97e+02  -3.8 2.53e+01    -  1.00e+00 1.04e-01f  1
  65  7.7549047e+02 2.50e-06

In [7]:
## Running Experiments for changing zeta ( multiplier for inventory level relative to production)
for ii in [0.05,0.1,0.15,0.2]:
    SOL_ = risk_neutral(DataPath='scens_two_stage/top_suppliers_scens/test_zeta{}scen.dat'.format(ii),
                        mipgap=0.00, 
                        timelimit=3600, 
                        nthreads=4, 
                        solver="ipopt", 
                        relaxed=False, 
                        displaySolver=True, 
                        warmstart=False,
                        WSOL={},  
                        verbose=False,)
    ## save solutions in pickle file 
    name = 'Results_RN_twostage/individual_scens/RN_zeta_scen{}'.format(ii) +'.pickle'
    with open(name, 'wb') as handle:
        pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1104403
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   199836

Total number of variables............................:   202032
                     variables with only lower bounds:   202032
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

  77  7.8274170e+02 4.69e-08 2.69e+02  -8.6 1.43e-02  -4.5 1.00e+00 9.23e-01f  1
  78  7.8274163e+02 4.69e-08 3.26e+03  -8.6 1.06e-03  -5.0 1.00e+00 7.31e-02f  1
  79  7.8274099e+02 4.69e-08 7.73e+02  -8.6 8.70e-04  -5.4 1.00e+00 7.80e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  7.8274094e+02 4.69e-08 2.49e+03  -8.6 1.75e-04  -5.0 1.00e+00 2.92e-01f  1
  81  7.8274094e+02 4.69e-08 3.44e+03  -8.6 1.16e-04  -4.6 1.00e+00 2.06e-02f  6
  82  7.8274083e+02 4.69e-08 2.94e+02  -8.6 1.12e-04  -5.1 1.00e+00 9.57e-01H  1
  83  7.8274083e+02 4.69e-08 5.82e-11  -8.6 1.10e-05  -5.5 1.00e+00 1.00e+00f  1
  84  7.8274083e+02 4.69e-08 5.82e-11  -8.6 2.40e-06  -5.1 1.00e+00 1.00e+00h  1
  85  7.8274083e+02 4.69e-08 4.37e-11  -8.6 2.53e-06  -5.6 1.00e+00 1.00e+00h  1
  86  7.8274083e+02 4.69e-08 4.37e-11  -8.6 7.65e-07  -5.2 1.00e+00 1.00e+00h  1
  87  7.8274083e+02 4.69e-08 3.64e-11  -8.6 2.83e-07  -4.7 1.00e+00 1.00e+00h  1
  88  7.8274083e+02 4.69e-08

  43  9.0821701e+02 3.54e-06 1.44e+02  -5.7 1.45e+02    -  2.75e-01 1.28e-01f  1
  44  8.9579909e+02 3.17e-06 1.49e+02  -5.7 1.29e+02    -  1.72e-01 1.01e-01f  1
  45  8.8084103e+02 2.74e-06 1.40e+02  -5.7 1.17e+02    -  1.85e-01 1.34e-01f  1
  46  8.7009605e+02 2.43e-06 1.51e+02  -5.7 1.02e+02    -  2.69e-01 1.11e-01f  1
  47  8.5939707e+02 2.12e-06 1.35e+02  -5.7 9.13e+01    -  1.42e-01 1.23e-01f  1
  48  8.5301483e+02 1.93e-06 1.39e+02  -5.7 8.00e+01    -  2.35e-01 8.39e-02f  1
  49  8.4874219e+02 1.81e-06 1.38e+02  -5.7 7.38e+01    -  1.70e-01 6.14e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  8.4169630e+02 1.61e-06 1.26e+02  -5.7 6.96e+01    -  1.63e-01 1.08e-01f  1
  51  8.3930508e+02 1.54e-06 1.28e+02  -5.7 6.19e+01    -  1.97e-01 4.09e-02f  1
  52  8.3353070e+02 1.37e-06 1.17e+02  -5.7 5.95e+01    -  1.72e-01 1.03e-01f  1
  53  8.2804465e+02 1.21e-06 1.06e+02  -5.7 5.37e+01    -  1.53e-01 1.09e-01f  1
  54  8.2547670e+02 1.14e-06

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  2.6824424e+02 3.22e+01 3.17e+03  -1.0 1.83e+02    -  3.18e-01 1.04e-01h  1
  11  2.7953410e+02 2.96e+01 3.48e+03  -1.0 1.57e+02    -  3.06e-01 7.34e-02h  1
  12  2.8535959e+02 2.84e+01 3.40e+03  -1.0 1.82e+02    -  4.83e-02 3.95e-02h  1
  13  3.3570699e+02 2.48e+01 2.36e+03  -1.0 1.77e+03    -  4.78e-04 1.10e-01h  1
  14  4.3587428e+02 1.74e+01 1.86e+03  -1.0 1.71e+03    -  2.74e-01 2.34e-01h  1
  15  4.7843136e+02 1.48e+01 2.01e+03  -1.0 1.30e+03    -  4.39e-01 1.34e-01h  1
  16  4.9733976e+02 1.37e+01 2.20e+03  -1.0 1.12e+03    -  2.45e-01 6.79e-02h  1
  17  6.1038964e+02 8.75e+00 1.15e+03  -1.0 1.04e+03    -  4.94e-03 2.94e-01h  1
  18  6.4025850e+02 7.78e+00 1.59e+03  -1.0 7.23e+02    -  5.03e-01 1.05e-01h  1
  19  8.5243331e+02 4.00e+00 7.94e+02  -1.0 7.36e+02    -  1.68e-02 4.00e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  9.2145839e+02 2.89e+00


 ---------- Objective function ----------
Min Cost: 782.7408293641475
Total Solving time: 238.866 [s]
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1104403
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   199836

Total number of variables............................:   202032
                     variables with only lower bounds:   202032
                variables with

  76  7.8275416e+02 4.69e-08 1.68e+03  -8.6 2.99e-02  -4.0 1.00e+00 5.22e-01f  1
  77  7.8274170e+02 4.69e-08 2.69e+02  -8.6 1.43e-02  -4.5 1.00e+00 9.23e-01f  1
  78  7.8274163e+02 4.69e-08 3.26e+03  -8.6 1.06e-03  -5.0 1.00e+00 7.31e-02f  1
  79  7.8274099e+02 4.69e-08 7.73e+02  -8.6 8.70e-04  -5.4 1.00e+00 7.80e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  7.8274094e+02 4.69e-08 2.49e+03  -8.6 1.75e-04  -5.0 1.00e+00 2.92e-01f  1
  81  7.8274094e+02 4.69e-08 3.44e+03  -8.6 1.16e-04  -4.6 1.00e+00 2.06e-02f  6
  82  7.8274083e+02 4.69e-08 2.94e+02  -8.6 1.12e-04  -5.1 1.00e+00 9.57e-01H  1
  83  7.8274083e+02 4.69e-08 5.82e-11  -8.6 1.10e-05  -5.5 1.00e+00 1.00e+00f  1
  84  7.8274083e+02 4.69e-08 5.82e-11  -8.6 2.40e-06  -5.1 1.00e+00 1.00e+00h  1
  85  7.8274083e+02 4.69e-08 4.37e-11  -8.6 2.53e-06  -5.6 1.00e+00 1.00e+00h  1
  86  7.8274083e+02 4.69e-08 4.37e-11  -8.6 7.65e-07  -5.2 1.00e+00 1.00e+00h  1
  87  7.8274083e+02 4.69e-08

In [ ]:
## Running Experiments increasing number of disruptions
stimes = []
sols = []
# nscen = [2,5,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,184]
nscen = [2*i for i in range(22,25)]
for ii in nscen:
    SOL_ = risk_neutral(DataPath='scens_two_stage/top_suppliers_scens/test{}scen.dat'.format(ii),
                        mipgap=0.00, 
                        timelimit=3600, 
                        nthreads=4, 
                        solver="ipopt", 
                        relaxed=False, 
                        displaySolver=True, 
                        warmstart=False,
                        WSOL={},  
                        verbose=False,)
    sols.append(SOL_['OBJ'])
    stimes.append(SOL_['STIME'])
    ## save solutions in pickle file 
    name = 'Results_RN_twostage/RN_SOL_scen{}'.format(ii) +'.pickle'
    with open(name, 'wb') as handle:
        pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  8820124
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:  1498770

Total number of variables............................:  1515240
                     variables with only lower bounds:  1515240
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

  77  3.5740980e+02 1.78e-06 5.34e+02  -5.7 4.59e+01    -  2.02e-01 1.03e-01f  1
  78  3.4957967e+02 1.65e-06 5.43e+02  -5.7 1.38e+02    -  2.53e-01 7.00e-02f  1
  79  3.4470343e+02 1.57e-06 5.25e+02  -5.7 1.28e+02    -  8.93e-02 4.77e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  3.3963986e+02 1.48e-06 5.26e+02  -5.7 1.23e+02    -  2.18e-01 5.13e-02f  1
  81  3.3520134e+02 1.41e-06 5.15e+02  -5.7 1.17e+02    -  1.54e-01 4.74e-02f  1
  82  3.3300047e+02 1.37e-06 5.17e+02  -5.7 1.11e+02    -  1.51e-01 2.47e-02f  1
  83  3.2682086e+02 1.27e-06 4.86e+02  -5.7 1.09e+02    -  1.35e-01 7.11e-02f  1
  84  3.2390673e+02 1.22e-06 4.77e+02  -5.7 1.02e+02    -  1.51e-01 3.61e-02f  1
  85  3.2079428e+02 1.17e-06 4.68e+02  -5.7 9.91e+01    -  1.91e-01 4.00e-02f  1
  86  3.1824922e+02 1.12e-06 4.57e+02  -5.7 9.60e+01    -  1.21e-01 3.40e-02f  1
  87  3.1551371e+02 1.08e-06 4.42e+02  -5.7 9.27e+01    -  8.06e-02 3.78e-02f  1
  88  3.1304266e+02 1.04e-06


 ---------- Objective function ----------
Min Cost: 244.68217693080229
Total Solving time: 15193.888 [s]
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  9215802
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:  1565382

Total number of variables............................:  1582584
                     variables with only lower bounds:  1582584
                variables w

  76  3.5122835e+02 7.81e-07 1.75e-08  -3.8 2.10e+00    -  1.00e+00 1.00e+00h  1
  77  3.4872480e+02 6.29e-07 2.65e+01  -5.7 3.09e+01    -  2.50e-01 1.75e-01f  1
  78  3.3724066e+02 5.48e-07 8.90e+01  -5.7 1.17e+02    -  3.59e-01 1.12e-01f  1
  79  3.2149884e+02 4.38e-07 7.14e+01  -5.7 1.06e+02    -  1.62e-01 1.74e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  3.1292564e+02 3.78e-07 8.07e+01  -5.7 8.80e+01    -  2.40e-01 1.14e-01f  1
  81  3.0351823e+02 3.12e-07 8.51e+01  -5.7 7.88e+01    -  2.92e-01 1.42e-01f  1
  82  2.9740841e+02 2.69e-07 8.31e+01  -5.7 6.79e+01    -  2.05e-01 1.07e-01f  1
  83  2.9405404e+02 2.46e-07 8.39e+01  -5.7 6.12e+01    -  1.76e-01 6.59e-02f  1
  84  2.8952811e+02 2.14e-07 8.15e+01  -5.7 5.74e+01    -  2.13e-01 9.51e-02f  1
  85  2.8618132e+02 1.91e-07 7.79e+01  -5.7 5.19e+01    -  1.55e-01 7.77e-02f  1
  86  2.8212866e+02 1.63e-07 7.42e+01  -5.7 4.79e+01    -  2.39e-01 1.02e-01f  1
  87  2.7765093e+02 1.31e-07

In [ ]:
## Running Experiments
# stimes = []
# sols = []
# # nscen = [2,5,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,184]
# nscen = [2*i for i in range(1,25)]
# for ii in nscen:
#     SOL_ = risk_neutral(DataPath='scenarios_top_suppliers/test{}scen.dat'.format(ii),
#                         mipgap=0.00, 
#                         timelimit=3600, 
#                         nthreads=4, 
#                         solver="ipopt", 
#                         relaxed=False, 
#                         displaySolver=True, 
#                         warmstart=False,
#                         WSOL={},  
#                         verbose=False,)
#     sols.append(SOL_['OBJ'])
#     stimes.append(SOL_['STIME'])
#     ## save solutions in pickle file 
#     name = 'results_RN/RN_SOL_scen{}'.format(ii) +'.pickle'
#     with open(name, 'wb') as handle:
#         pickle.dump(SOL_, handle, protocol=pickle.HIGHEST_PROTOCOL)
    